In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
train_df = pd.read_csv('/kaggle/input/tabular-playground-series-jan-2022/train.csv')
test_df = pd.read_csv('/kaggle/input/tabular-playground-series-jan-2022/test.csv')

In [ ]:
train_df.head(10)

In [ ]:
train_df.shape

In [ ]:
train_df.isnull().sum()

In [ ]:
train_df['country'].unique()

In [ ]:
train_df['store'].unique()

In [ ]:
train_df['product'].unique()

In [ ]:
train_df = train_df.set_index('row_id')

In [ ]:
train_df.info()

In [ ]:
train_df['date'] = pd.to_datetime(train_df['date'])

In [ ]:
train_df.info()

In [ ]:
train_df.head()

In [ ]:
train_df['month'] = train_df['date'].dt.month

In [ ]:
train_df['day_of_week'] = train_df['date'].dt.dayofweek

In [ ]:
train_df.head()

In [ ]:
sns.boxplot(data=train_df,x='store',y='num_sold')

In [ ]:
sns.boxplot(data=train_df,x='product',y='num_sold')

In [ ]:
plt.figure( figsize= (10,8))
sns.boxplot(data=train_df,x='product',y='num_sold',hue='country')

Norway seems to be leading in sales

In [ ]:
train_df.groupby('month')['num_sold'].sum().plot(kind='barh')

In [ ]:
train_df.groupby('day_of_week')['num_sold'].sum().plot(kind='barh')

In [ ]:
## Dropping the date column as important month and day features extracted
train_df = train_df.drop('date',axis=1)

In [ ]:
train_df.head()

In [ ]:
y= train_df['num_sold']
X= train_df.drop('num_sold',axis=1)

In [ ]:
X.head()

In [ ]:
## one hot encoding of the dataset
category = ['country','store','product']
X = pd.get_dummies(data=X,columns=category,drop_first=True)

In [ ]:
from sklearn.model_selection import train_test_split

X_train,X_validate,y_train,y_validate = train_test_split(X,y,test_size=0.2,random_state=42)

In [ ]:
print(X_train.shape)
print(X_validate.shape)
print(y_train.shape)
print(y_validate.shape)

In [ ]:
X_train.columns

In [ ]:
## transforming the test data in similer terms
test_df = test_df.set_index('row_id')
test_df['date'] = pd.to_datetime(test_df['date'])
test_df['month'] = test_df['date'].dt.month
test_df['day_of_week'] = test_df['date'].dt.dayofweek

In [ ]:
test_df.drop('date',axis=1,inplace=True)

In [ ]:
category = ['country','store','product']
test_df = pd.get_dummies(data=test_df,columns=category)

In [ ]:
test_df.columns

In [ ]:
## dropping country_Finland,store_KaggleMart,product_Kaggle Hat
test_df.drop(['country_Finland','store_KaggleMart','product_Kaggle Hat'],axis=1,inplace=True)

In [ ]:
test_df.columns

In [ ]:
test_df.shape

## Model Building

### 1. Based on Linear Regression -Base model

In [ ]:
import statsmodels.api as sm

In [ ]:
X_train_c = sm.add_constant(X_train)
model  = sm.OLS(y_train,X_train_c)
results = model.fit()
results.summary()

Since probability for all features is less than 0.05 , all features are significant

In [ ]:
from statsmodels.stats.outliers_influence import variance_inflation_factor
vifs = [variance_inflation_factor(X_train,i) for i in range(X_train.shape[1])]
vif_df = pd.DataFrame({'cols':X_train.columns,'vif':vifs})
vif_df

Values are well within range. So no multicollinearity

In [ ]:
y_validate_predict = results.predict(sm.add_constant(X_validate))

In [ ]:
from sklearn.metrics import r2_score
print('R2 score on validation set is ',r2_score(y_validate,y_validate_predict))
validation_scores = {}
validation_scores['lr'] = r2_score(y_validate,y_validate_predict)

Considering this Linear Regression one as a base model ,go forward with Other type of models

In [ ]:
from sklearn.metrics import mean_squared_error
mse = mean_squared_error(y_validate,y_validate_predict)
rmse = mse**0.5
print('Root mean squared error is ', rmse)

In [ ]:
def smape(A, F):
    return 100/len(A) * np.sum(2 * np.abs(F - A) / (np.abs(A) + np.abs(F)))

In [ ]:
print(smape(y_validate,y_validate_predict))

In [ ]:
test_df['pred'] = results.predict(sm.add_constant(test_df))

In [ ]:
test_df

In [ ]:
## for rows with negative values , convert to 0
test_df['pred'] = test_df['pred'].apply(lambda x : 0 if x<0 else x)

In [ ]:
test_df.reset_index(inplace=True)

In [ ]:
test_df.rename(columns={'pred':'num_sold'},inplace=True)

In [ ]:
#test_df[['row_id','num_sold']].to_csv('./submission.csv',index=False)

### 2. Decision tree based model

1) Base Decision Tree

In [ ]:
from sklearn.tree import DecisionTreeRegressor
dt = DecisionTreeRegressor(random_state=42)


In [ ]:
dt.fit(X_train,y_train)
y_validate_pred_dt = dt.predict(X_validate)
y_train_pred_dt = dt.predict(X_train)

In [ ]:
print('R2 score for Validation set is ',r2_score(y_validate,y_validate_pred_dt))
print('R2 score for Train set is',r2_score(y_train,y_train_pred_dt))

2) Hyper parameter tuning

In [ ]:
%%time
from sklearn.model_selection import GridSearchCV
dt = DecisionTreeRegressor(random_state=42)
paramgrid = {
    'max_depth':[5,10,15,20],
    'min_samples_split':[20,40,50]
}
grid = GridSearchCV(dt,param_grid=paramgrid,scoring='r2',return_train_score=True,verbose=1,n_jobs=-1)

In [ ]:
%%time
grid.fit(X_train,y_train)

In [ ]:
grid.best_params_

In [ ]:
grid.best_score_

In [ ]:
dt_model = grid.best_estimator_

In [ ]:
y_validate_pred = dt_model.predict(X_validate)
smape(y_validate,y_validate_pred)

In [ ]:
test_df['dt_pred'] = dt_model.predict(test_df[['month', 'day_of_week', 'country_Norway', 'country_Sweden',
       'store_KaggleRama', 'product_Kaggle Mug', 'product_Kaggle Sticker']])

In [ ]:
test_df.rename(columns={'num_sold':'lr_pred'},inplace=True)

In [ ]:
#test_df[['row_id','dt_pred']].rename(columns={'dt_pred':'num_sold'}).to_csv('./submission.csv',index=False)

### 3. Ensemble Training - Random Forest

1.) Base Model

In [ ]:
from sklearn.ensemble import RandomForestRegressor

In [ ]:
rf_base = RandomForestRegressor(random_state=42)

In [ ]:
rf_base.fit(X_train,y_train)

In [ ]:
y_validate_rfbase_pred = rf_base.predict(X_validate)

In [ ]:
print('Base RF model r2score is ',r2_score(y_validate,y_validate_rfbase_pred))

2.) Hyperparameter Tuning

In [ ]:
paramgrid={
    'max_depth':[5,10,15,20],
    'min_samples_split':[20,40,50],
    'n_estimators':[20,30,40],
    'max_features':[3,4,5]
}

In [ ]:
rf = RandomForestRegressor(random_state=42)

In [ ]:
grid = GridSearchCV(rf,param_grid=paramgrid,n_jobs=-1,verbose=1,scoring='r2')

In [ ]:
%%time
grid.fit(X_train,y_train)

In [ ]:
## params of optimised model
grid.best_params_

In [ ]:
rf_optimum = grid.best_estimator_

In [ ]:
y_validate_rfopt_pred = rf_optimum.predict(X_validate)

In [ ]:
print('Optimized RF model r2score is ',r2_score(y_validate,y_validate_rfopt_pred))

In [ ]:
## use the optimized model for prediction on test_df
test_df['rf_pred'] = rf_optimum.predict(test_df[['month', 'day_of_week', 'country_Norway', 'country_Sweden',
       'store_KaggleRama', 'product_Kaggle Mug', 'product_Kaggle Sticker']])

In [ ]:
## correct negative values to 0
test_df['rf_pred'] = test_df['rf_pred'].apply(lambda x: 0 if x<0 else x)

In [ ]:
test_df[['row_id','rf_pred']].rename(columns={'rf_pred':'num_sold'}).to_csv('./submission.csv',index=False)